<h2>Slitmask module

<b><i> This is not a developed notebook and just gives some test usage of slitmask routines, which are not fully developed/tested

In [1]:
from pyvista import imred, tv, stars, slitmask, image
import pdb
import copy
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.table import vstack

In [2]:
%matplotlib tk
t=tv.TV()

<h3> Acquisition </h3>

In [3]:
red=imred.Reducer('KOSMOS',dir='/home/holtz/raw/apo/UT221221_cleaned',verbose=False)                                                         

In [4]:
red.log()

FILE,DATE-OBS,OBJNAME,RA,DEC,EXPTIME
bytes43,bytes26,bytes16,bytes16,bytes16,bytes16
bias.0001.fits,2022-12-21T07:03:19.199073,MUSSES2022D,7:44:26.48,44:17:03.60,0
bias.0002.fits,2022-12-21T07:04:12.915674,MUSSES2022D,7:44:26.48,44:17:03.60,0
bias.0003.fits,2022-12-21T07:05:07.210469,MUSSES2022D,7:44:26.48,44:17:03.60,0
bias.0004.fits,2022-12-21T07:06:00.729529,MUSSES2022D,7:44:26.48,44:17:03.60,0
bias.0005.fits,2022-12-21T07:06:54.510369,MUSSES2022D,7:44:26.48,44:17:03.60,0
mask63_arc_Ne_red_0p5s.0017.fits,2022-12-21T09:04:31.086764,eFEDS197p1,8:36:57.96,3:01:04.69,0.5
mask63_arc_Ne_red_0p5s.0018.fits,2022-12-21T09:05:25.327441,eFEDS197p1,8:36:57.96,3:01:04.69,0.5
mask63_arc_Ne_red_0p5s.0019.fits,2022-12-21T09:06:19.582805,eFEDS197p1,8:36:57.96,3:01:04.69,0.5
mask63_arc_Ar_red_0p5s.0020.fits,2022-12-21T09:10:43.446486,eFEDS197p1,8:36:57.96,3:01:04.69,0.5


Read in image with mask in, and image without mask

In [5]:
mask=red.reduce(51)
t.clear()
t.tv(mask,max=3000)

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


In [6]:
nomask=red.reduce(52)
t.tv(nomask,max=30000)

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Do CR rejection on mask image to avoid spurious detections

In [7]:
maskcr=red.crrej(mask,crbox=[5,5])

Use findholes() to identify the holes in the mask. May need to adjust thresh= for brightness threshold, and n= for number of clusters to detect.

In [8]:
holes=slitmask.findholes(maskcr.data,thresh=1500,n=29)
t.clear()
t.tv(maskcr)
stars.mark(t,holes,color='b',exit=True,rad=10)

y,x
float64,float64
1847.4569536423842,1388.3741721854306
2194.5,728.0
1864.5,616.0
1829.5,1008.0
2627.5353535353534,489.050505050505
1274.6354166666667,799.4965277777778
1998.4494773519164,1790.212543554007
2221.3120567375886,910.5177304964539
1621.7963636363636,1104.3454545454545


Get design from KMS file and see if we have correct numbers of slits/holes identified

In [9]:
kmsfile='eFEDS197_pointing2.1.kms'
targets = slitmask.read_kms(kmsfile,sort='YMM')
print(len(targets),len(holes))
targets

15 15


ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
str7,str5,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64
TARG114,NN,STRAIGHT,4.0,4.0,0.0,83654.162,30638.76,0.683,0.683,-6.272,-33.306
TARG111,NN,STRAIGHT,4.0,4.0,0.0,83645.269,30628.86,0.683,0.683,16.462,-31.616
TARG109,Obj9,STRAIGHT,2.0,10.0,0.0,83702.29,30538.62,0.341,1.707,-27.05,-23.042
TARG101,Obj1,STRAIGHT,2.0,10.0,0.0,83649.106,30504.92,0.341,1.707,6.652,-17.29
TARG103,Obj3,STRAIGHT,2.0,10.0,0.0,83654.799,30436.19,0.341,1.707,-7.901,-12.387
TARG102,Obj2,STRAIGHT,2.0,10.0,0.0,83638.93,30417.69,0.341,1.707,32.666,-9.23
TARG104,Obj4,STRAIGHT,2.0,10.0,0.0,83655.258,30349.0,0.341,1.707,-9.075,-4.333
TARG105,Obj5,STRAIGHT,2.0,10.0,0.0,83648.518,30323.98,0.341,1.707,8.155,-0.063
TARG106,Obj6,STRAIGHT,2.0,10.0,0.0,83644.95,30258.99,0.341,1.707,17.277,4.202


Now sort the identified holes from left to right and match them
with the correspondingly sorted design holes to identify which ones are alignment holes.

In [12]:
# sort holes by x coordinate
holes.sort('x')

# loop through holes and targets to find alignment holes
align=[]
for i,(hole,target) in enumerate(zip(holes,targets)) :
    #print(hole['x'],target['YMM'],target['NAME'])
    if target['NAME'] == 'NN' : align.append(i)

# Display the alignment holes
stars.mark(t,holes[align],color='r',rad=20,exit=True)

#make table of the measured alignment hole positions
h=holes[align]
h

y,x
float64,float64
2158.0,255.0
1643.0,292.5
1255.0,1179.0
1992.5185185185185,1257.530864197531
2451.4347826086955,1525.7782608695652


Now display image and mark stars that corresonds to the alignment holes in the mask.

In [13]:
t.tv(nomask,max=30000)
s=stars.mark(t,rad=7)

# sort these by x coorindate to match order of alignment holes
s.sort('x')
s

Hit c near desired star(s) to get centroid position
    i to use integer position of cursor
    n to get ID of nearest star
    q or e to quit


id,x,y,MJD,EXPTIME,FILTER,AIRMASS
int32,float32,float32,float64,float32,bytes1,float32
1,221.95,2205.29,59934.466355,30.0,,1.244
2,260.76,1691.18,59934.466355,30.0,,1.244
5,1146.17,1306.42,59934.466355,30.0,,1.244
3,1223.02,2043.20,59934.466355,30.0,,1.244
4,1489.37,2506.04,59934.466355,30.0,,1.244


Now use fit() to get the rotation and translation. Output the distance between the measured object and the fit one to ensure that the identification is correct and the solution is reasonable.

In [14]:
slitmask.fit(h,s)

Similarity Transform: 
Rotation (degrees) :  0.22590711437158215
Translation (arcsec)  [-8.87385559 13.10466513]
Scale  0.9991657956514659
 X       Y       mod-obs 
  -769.0   110.0    1.19
  -731.5  -405.0    0.45
   155.0  -793.0    1.57
   233.5   -55.5    1.09
   501.8   403.4    2.18


(0.22590711437158215, array([-8.87385559, 13.10466513]), 0.9991657956514659)

<h3>Reduction/extraction

In [4]:
from pyvista import imred, slitmask, tv, spectra, image
import numpy as np
import matplotlib.pyplot as plt
import copy
import pdb
%matplotlib tk

t=tv.TV()

In [3]:
red=imred.Reducer('KOSMOS',dir='/home/holtz/raw/apo/UT221221_cleaned',verbose=False)                                                         
flat = red.reduce(71)
arc = red.reduce(70)
arcs= red.sum([70,67])

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Find slit edges from flat, and fit polynomials to locations

In [5]:
trace=spectra.Trace(transpose=True)
t.tvclear()
bottom,top = trace.findslits(flat,display=t,thresh=0.5,sn=True)

Plot shape of slit edges relative to center position, just for information

In [6]:
x=np.arange(4096)
plt.figure()
for l in bottom :
    plt.plot(x,l(x)-l(2048))

Read in the slitmask file, which we'll use to get object names and slit locations to help with wavelength solution.

If the number of slitlets found doesn't match the number of targets, you'll need to go back and adjust the threshold to find the correct slitlets, or else modify the targets table below so that they match.

In [6]:
kmsfile='eFEDS197_pointing2.1.kms'
targets = slitmask.read_kms(kmsfile,sort='YMM')
if len(targets) != len(bottom) : 
    print('ERROR, number of identified slits does not match number of targets')
targets

ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
str7,str5,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64
TARG114,NN,STRAIGHT,4.0,4.0,0.0,83654.162,30638.76,0.683,0.683,-6.272,-33.306
TARG111,NN,STRAIGHT,4.0,4.0,0.0,83645.269,30628.86,0.683,0.683,16.462,-31.616
TARG109,Obj9,STRAIGHT,2.0,10.0,0.0,83702.29,30538.62,0.341,1.707,-27.05,-23.042
TARG101,Obj1,STRAIGHT,2.0,10.0,0.0,83649.106,30504.92,0.341,1.707,6.652,-17.29
TARG103,Obj3,STRAIGHT,2.0,10.0,0.0,83654.799,30436.19,0.341,1.707,-7.901,-12.387
TARG102,Obj2,STRAIGHT,2.0,10.0,0.0,83638.93,30417.69,0.341,1.707,32.666,-9.23
TARG104,Obj4,STRAIGHT,2.0,10.0,0.0,83655.258,30349.0,0.341,1.707,-9.075,-4.333
TARG105,Obj5,STRAIGHT,2.0,10.0,0.0,83648.518,30323.98,0.341,1.707,8.155,-0.063
TARG106,Obj6,STRAIGHT,2.0,10.0,0.0,83644.95,30258.99,0.341,1.707,17.277,4.202


In [7]:
# if you want subset of slitlets, e.g. ignoring alignments stars, select them here
gd=np.where(targets['NAME']!='NN')[0]     # or, e.g., gd=[2,3,4,7,9]
gdtrace=copy.deepcopy(trace)
gdtrace.model = [trace.model[i] for i in gd]
gdtrace.rows = [trace.rows[i] for i in gd]
trace=copy.deepcopy(gdtrace)
targets=targets[gd]
vars(trace)

{'type': 'Polynomial1D',
 'degree': 2,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-50, 50),
 'model': [Polynomial([455.39492819,   6.86250423, -10.5109358 ], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([584.86096608,   6.98188667,  -7.20510802], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([695.30089454,   7.03335661,  -4.50728817], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([769.89113187,   7.06478212,  -2.53785573], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([ 8.78061651e+02,  7.09096738e+00, -1.05166117e-01], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([975.73794736,   7.10712024,   2.25961684], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1072.80140623,    7.06138006,    4.58015777], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1356.17765073,    7.06397733,   11.243

Using the derived traces, extract the slitlets for arcs

In [9]:
arcec=trace.extract2d(arcs,display=t)

extracting: 
 455-494
 585-624
 695-734
 770-809
 878-917
 976-1015
 1073-1112
 1356-1395
 1642-1680
 1758-1796
  See extraction window(s). Hit space bar to continue....


Add XMM and YMM for each slit to headers of each extracted image. They need to match so you get the right values!

In [10]:
for arc,target in zip(arcec,targets) : 
    arc.header['XMM'] = target['XMM']
    arc.header['YMM'] = target['YMM']

Now loop through each extracted arc to do wavelength calibration. This requires a little effort because the change in the location of the slit relative to the default saved wavelength calibration is significant enough that it can be a challenge to automatically find the lines, since the change in spectrum is more than a simple shift (and, in fact, more than a shift + dispersion change). 

However, a simple shift is usually enough to identify some of the lines, and these can be used to bootstrap the wavelength solution; the initial identification is easier if you use an estimate of the shift from the mask design XMM.

You can use identify() to do the iteration. On the first pass, only central lines may be correctly identified. Use 'l' and 'r' to remove lines to the left and right of the identified lines. Then use 'i' to iterate, i.e., allow it to re-identify lines (i just returns True to allow you to iterate). When happy with solution, use ' ' to move onto the final 2D wavelength calibration.

You can really help this process if you supply an initial wavelength calibration (a pyvista WaveCal object) that was done using the same lamp(s) as your arc exposures (here, using KOSMOS_red_waves.fits'), and using a master line list that corresponds to these lamp(s) (here, using ne.dat). If you choose another WaveCal to start from, you may need to get a correct approximate relation for the shift from the reference spectrum as a function of XMM. For KOSMOS, seems like -22.5(XMM) gives a rough pixel shift from a center slit location, -22.5(XMM-24.44) for a low slit location.

The next cell just shows how the simple shift fails, even using the XMM for each slit to shift: all of the arc lines don't match up with just a translation.

In [28]:
plt.figure()
wav2=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
plt.plot(wav2.spectrum[0])
for i, arc in enumerate(arcec[0:1]) :
    shift=(arc.header['XMM']*-22.5)
    plt.plot(arc.data[19][int(shift):]*30)
    print(shift)
    plt.draw()

  rms:    0.177 Angstroms (50 lines)
608.625


The wavelength calibration for each slitlet will be saved. Since this is probably the most time-consuming part, you could use saved one if you only want to redo a couple of them (sometimes, you hit the wrong key and want to do one over!)

In [100]:
clobber=True          # set to False if you want to use any saved ones
for i,(arc,targ) in enumerate(zip(arcec,targets)) :
    wavname = 'wav_{:s}.fits'.format(targ['NAME'])
    if clobber or not os.path.exists(wavname) :
        wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
        wav.fit(degree=3)
        nrow=arc.shape[0]
    
        # get initial guess at shift from reference using XMM (KOSMOD red low!)
        shift=int(arc.header['XMM']*-22.5) # +550  #-wav.pix0)
        lags=np.arange(shift-20,shift+20)

        iter = True
        while iter :
            iter = wav.identify(arc[nrow//2],plot=True,plotinter=True,lags=lags,thresh=10,file='henear.dat')
            lags=np.arange(-50,50)
            plt.close()

        bd= np.where(wav.weights<0.5)[0]
        print(wav.waves[bd])
        # Do the 2D wavelength solution, sampling 10 locations across slitlet
        wav.degree=5
        wav.identify(arc,plot=True,nskip=nrow//10,thresh=10)
        plt.close()
        wav.write(wavname)
        wav.add_wave(arc)
        t.tv(wav.correct(arc,arc.wave[nrow//2]))

  rms:    0.177 Angstroms (50 lines)
  rms:    0.177 Angstroms (50 lines)
  cross correlating with reference spectrum using lags between:  -169 -130
  Derived pixel shift from input wcal:  [-151.77177119]


/home/holtz/git/pyvista/python/pyvista/spectra.py:708: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  See identified lines.
  rms:   27.450 Angstroms (45 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:   27.450 Anstroms
  input from plot window...
  rms:    2.183 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -50 49
  Derived pixel shift from input wcal:  [6.53867502e-08]
  See identified lines.
  rms:    8.925 Angstroms (48 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:    8.925 Anstroms
  input from plot window...
  rms:    1.346 Anstroms
  input from plot wi

Now set up routine to reduct/extract science frames

In [11]:
def multi_extract2d(red,trace,targets,image,bias=None,dark=None,flat=None,display=None,crbox='lacosmic',crsig=10,
                    plot=False,rad=5) :

    if display is not None : display.clear()
    # basic image read and reduction
    imcr=red.reduce(image,bias=bias,dark=dark,flat=flat,crbox=crbox,display=display,crsig=crsig)

    # 2D extraction
    out = trace.extract2d(imcr,display=display)

    # loop over each desired slitlet, get wavelength shift from skylines, undistort to central wavelengths
    diff=[]
    for i,(o,targ) in enumerate(zip(out,targets)) :
        wav=spectra.WaveCal('./wav_{:s}.fits'.format(targ['NAME']))
        orig=wav.model.c0_0
        wav.add_wave(o)

        # set rows to use for skyline
        nrows = o.shape[0]
        rows = [x for x in range(0,nrows) if np.abs(x-nrows//2)>rad]
        wav.skyline(o,thresh=10,rows=rows,plot=plot)
        
        if plot : plt.close()
        wav.add_wave(o)
        if display is not None : display.tv(o)
        out[i]=wav.correct(o,o.wave[nrows//2])
        if display is not None : display.tv(out[i])
        name = out[i].header["FILE"].split(".")[0] 
        out[i].write('{:s}_{:s}_2d.fits'.format(name,targ['NAME']))
        diff.append(wav.model.c0_0-orig)

    print('shifts: ',diff)
    return out


In [12]:
out=multi_extract2d(red,trace,targets,54,display=None)

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
extracting: 
 455-494
 585-624
 695-734
 770-809
 878-917
 976-1015
 1073-1112
 1356-1395
 1642-1680
 1758-1796
  rms:    0.319
rejecting 6 points from 243 total: 
  rms:    0.319
rejecting 6 points from 243 total: 
  rms:    0.341
rejecting 0 points from 187 total: 

appending uncertainty
appending bitmask
appending wave
  rms:    0.221
rejecting 4 points from 555 total: 
  rms:    0.221
rejecting 4 points from 555 total: 


/home/holtz/git/pyvista/python/pyvista/spectra.py:708: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  rms:    0.850
rejecting 3 points from 324 total: 
  rms:    0.629
rejecting 4 points from 324 total: 
  rms:    0.613
rejecting 4 points from 324 total: 

appending uncertainty
appending bitmask
appending wave
  rms:    0.358
rejecting 11 points from 309 total: 
  rms:    0.358
rejecting 11 points from 309 total: 
  rms:    0.617
rejecting 4 points from 266 total: 
  rms:    0.451
rejecting 4 points from 266 total: 

appending uncertainty
appending bitmask
appending wave
  rms:    0.286
rejecting 7 points from 249 total: 
  rms:    0.286
rejecting 7 points from 249 total: 
  rms:   10.756
rejecting 1 points from 293 total: 
  rms:    1.133
rejecting 1 points from 293 total: 

appending uncertainty
appending bitmask
appending wave
  rms:    0.349
rejecting 13 points from 313 total: 
  rms:    0.349
rejecting 13 points from 313 total: 
  rms:  320.534
rejecting 1 points from 251 total: 
  rms:    1.012
rejecting 1 points from 251 total: 

appending uncertainty
appending bitmask
appendi

As desired, move on to 1D extraction

In [15]:
def multi_extract1d(spec2d) :
    def model(x) :
        return x*0.

    fig=plt.figure()
    spec1d=[]
    for i in range(len(spec2d)) :
        trace1 = spectra.Trace(transpose=False)
        trace1.rows = [0,spec2d[i].data.shape[0]]
        trace1.index = [0]
        peak,ind = trace1.findpeak(spec2d[i],thresh=10,sort=True)
        if len(peak) > 0:
            def model(x) :
                return x*0. + peak[0]
            trace1.model = [model]
            spec=trace1.extract(spec2d[i],rad=4,back=[[-10,-5],[5,10]],display=None)
            plt.figure(fig)
            spec.wave = out[i].wave[peak]
            print(spec.wave[0].shape,spec.data[0].shape)
        
            plt.plot(spec.wave[0],spec.data[0])
            spec1d.append(spec)
        else :
            print('no peak found for slit: ',i)
    return spec1d

In [16]:
spec1d = multi_extract1d(out)

looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [8, 34]
aperture/fiber:  [0, 1]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [19]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [19]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [21, 33]
aperture/fiber:  [0, 1]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [18]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [20]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [20]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
